In [73]:
import timeit
start = timeit.default_timer()

import pandas as pd
import numpy as np
import random
import copy
import math
from collections import defaultdict

randomlist = []
df_demand = {}
random.seed(11) # Use the same random seed
for i in range(0,20):   # 20 depots
    n = random.randint(50,150) # Depot Capacity
    randomlist.append(n)

for i in range(20):
    df_demand['D'+str(i+1)] = randomlist[i]

sum = 0
for (k,v) in df_demand.items():
    sum = sum + v

df_supply = {}
for i in range(sum):
    df_supply[i] = 1.0

np.random.seed(11) # Use the same random seed
df_dkm = pd.DataFrame(np.random.uniform(0.5,40, size=(sum,20))).astype(float)

random.seed(11) # Use the same random seed
age_b = []
for i in range(sum):
    n = random.randint(1,3) #Only to generate integer values inclusive of 1 & 3
    age_b.append(n)

age = {}
for i in range(len(age_b)):
    age[i] = age_b[i]

kpl = []
co2 = []
random.seed(11) # Use the same random seed
for (k,v) in age.items():
    if (v==1):
        n = random.uniform(5.1,6)
        m = round(random.uniform(515, 524), 2)
        kpl.append(n)
        co2.append(m)
    if (v==2):
        n = random.uniform(4.1,5)
        m = round(random.uniform(525, 534), 2)
        kpl.append(n)
        co2.append(m)
    if (v==3):
        n = random.uniform(3.1,4)
        m = round(random.uniform(535, 540), 2)
        kpl.append(n)
        co2.append(m)

fc_kpl = {}
for i in range(sum):
    fc_kpl[i] =(111/ kpl[i])

co2_c = {}
for i in range(sum):
    co2_c[i] =(0.00118642*co2[i])

random.seed(11) # Use the same random seed
doc = {}
for i in range(0,20):
    n = random.uniform(50,100)
    doc[i] = n

df_tdkom = pd.DataFrame(np.zeros(df_dkm.shape))
for i in range(len(df_dkm.columns)):
    for j in range(len(df_dkm.index)):
        df_tdkom[i][j] = df_dkm[i][j]*fc_kpl[j] + df_dkm[i][j]*co2_c[j] + doc[i]

In [74]:
df_tdkom.columns = [("D"+str(i)) for i in range(1,21)]
dk = df_tdkom.transpose()
costs_t = dk.to_dict()
#costs_t

costs1=copy.deepcopy(costs_t)
cols = sorted(df_demand.keys())

In [75]:
for i in df_supply:
    for j in df_demand:
        costs_t[i][j] = costs_t[i][j]*min(df_supply[i],df_demand[j])

In [76]:
res = dict((k, defaultdict(int)) for k in costs_t)

In [77]:
while (costs_t):
    sub_dict = {}
    min_value_n = 10000
    for i in df_supply:
        sub_dict = costs_t[i]
        #print(sub_dict)
        min_value, min_key = min((value, key) for key, value in sub_dict.items())
        #print(min_value, min_key)
        if min_value <= min_value_n:
            #print(min_value, min_value_n)
            min_value_n = min_value
            min_key_n = min_key
            sub_key = i
            continue
        #else:
        #    break


    #print(sub_key, min_key_n, min_value_n)

    res[sub_key][min_key_n] = min(df_supply[sub_key],df_demand[min_key_n])
    df_demand[min_key_n]-= res[sub_key][min_key_n]
    df_supply[sub_key]-= res[sub_key][min_key_n]

    if df_demand[min_key_n] == 0:
        del df_demand[min_key_n]
        for i in df_supply:
            del costs_t[i][min_key_n]

    if df_supply[sub_key] == 0:
        del df_supply[sub_key]
        del costs_t[sub_key]

In [78]:
df_supply

{}

In [79]:
cost = 0
for g in sorted(costs1):
    # print (g, "\t",)
    for n in cols:
        y = res[g][n]
        if y != 0:
            pass
            # print (y,)
        cost += y * costs1[g][n]
        # print ("\t",)
    # print()
print ("\n\nTotal Cost = ", cost)



Total Cost =  331475.10200162407
